This is a companion notebook for the book [Deep Learning with Python, Second Edition](https://www.manning.com/books/deep-learning-with-python-second-edition?a_aid=keras&a_bid=76564dff). For readability, it only contains runnable code blocks and section titles, and omits everything else in the book: text paragraphs, figures, and pseudocode.

**If you want to be able to follow what's going on, I recommend reading the notebook side by side with your copy of the book.**

This notebook was generated for TensorFlow 2.6.

## Neural style transfer

### The content loss

### The style loss

### Neural style transfer in Keras

**Getting the style and content images**

In [1]:
from tensorflow import keras

# 待变换图像
base_image_path = keras.utils.get_file(
    "sf.jpg", origin="https://img-datasets.s3.amazonaws.com/sf.jpg")
# 风格图像
style_reference_image_path = keras.utils.get_file(
    "starry_night.jpg", origin="https://img-datasets.s3.amazonaws.com/starry_night.jpg")

# 生成图像的尺寸
original_width, original_height = keras.utils.load_img(base_image_path).size
img_height = 400
img_width = round(original_width * img_height / original_height)

943128/943128 [==============================] - 1s 1us/step


**Auxiliary functions**

In [2]:
import numpy as np

# 加载图片
def preprocess_image(image_path):
    img = keras.utils.load_img(
        image_path, target_size=(img_height, img_width))
    img = keras.utils.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = keras.applications.vgg19.preprocess_input(img)
    return img

# 将Numpy数组转换为有效图像|
def deprocess_image(img):
    img = img.reshape((img_height, img_width, 3))
    img[:, :, 0] += 103.939
    img[:, :, 1] += 116.779
    img[:, :, 2] += 123.68
    img = img[:, :, ::-1]
    img = np.clip(img, 0, 255).astype("uint8")
    return img

**Using a pretrained VGG19 model to create a feature extractor**

In [3]:
# 构建VGG19模型，加载预训练的ImageNet权重
model = keras.applications.vgg19.VGG19(weights="imagenet", include_top=False)

# 返回每个目标层的激活值
outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])
feature_extractor = keras.Model(inputs=model.inputs, outputs=outputs_dict)

80134624/80134624 [==============================] - 21s 0us/step


**Content loss**

In [4]:
def content_loss(base_img, combination_img):
    return tf.reduce_sum(tf.square(combination_img - base_img))

**Style loss**

In [5]:
def gram_matrix(x):
    x = tf.transpose(x, (2, 0, 1))
    features = tf.reshape(x, (tf.shape(x)[0], -1))
    gram = tf.matmul(features, tf.transpose(features))
    return gram

def style_loss(style_img, combination_img):
    S = gram_matrix(style_img)
    C = gram_matrix(combination_img)
    channels = 3
    size = img_height * img_width
    return tf.reduce_sum(tf.square(S - C)) / (4.0 * (channels ** 2) * (size ** 2))

**Total variation loss**

In [6]:
# 促使生成图像具有空间连续性，避免得到过度像素化的结果
def total_variation_loss(x):
    a = tf.square(
        x[:, : img_height - 1, : img_width - 1, :] - x[:, 1:, : img_width - 1, :]
    )
    b = tf.square(
        x[:, : img_height - 1, : img_width - 1, :] - x[:, : img_height - 1, 1:, :]
    )
    return tf.reduce_sum(tf.pow(a + b, 1.25))

**Defining the final loss that you'll minimize**

In [7]:
# 用于风格损失的层
style_layer_names = [
    "block1_conv1",
    "block2_conv1",
    "block3_conv1",
    "block4_conv1",
    "block5_conv1",
]
# 用于内容损失的层
content_layer_name = "block5_conv2"
# 设置总变差损失的权重
total_variation_weight = 1e-6
# 风格损失的权重
style_weight = 1e-6
# 内容损失的权重
content_weight = 2.5e-8

def compute_loss(combination_image, base_image, style_reference_image):
    input_tensor = tf.concat(
        [base_image, style_reference_image, combination_image], axis=0
    )
    features = feature_extractor(input_tensor)
    loss = tf.zeros(shape=())
    layer_features = features[content_layer_name]
    base_image_features = layer_features[0, :, :, :]
    combination_features = layer_features[2, :, :, :]
    loss = loss + content_weight * content_loss(
        base_image_features, combination_features
    )
    # 添加风格损失
    for layer_name in style_layer_names:
        layer_features = features[layer_name]
        style_reference_features = layer_features[1, :, :, :]
        combination_features = layer_features[2, :, :, :]
        style_loss_value = style_loss(
          style_reference_features, combination_features)
        loss += (style_weight / len(style_layer_names)) * style_loss_value

    loss += total_variation_weight * total_variation_loss(combination_image)
    return loss

**Setting up the gradient-descent process**

In [8]:
import tensorflow as tf

@tf.function
def compute_loss_and_grads(combination_image, base_image, style_reference_image):
    with tf.GradientTape() as tape:
        loss = compute_loss(combination_image, base_image, style_reference_image)
    grads = tape.gradient(loss, combination_image)
    return loss, grads

# 初始值为100，每次减小4%
optimizer = keras.optimizers.SGD(
    keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=100.0, decay_steps=100, decay_rate=0.96
    )
)

base_image = preprocess_image(base_image_path)
style_reference_image = preprocess_image(style_reference_image_path)
# 在训练过程总会不断更新图像
combination_image = tf.Variable(preprocess_image(base_image_path))

iterations = 4000
for i in range(1, iterations + 1):
    loss, grads = compute_loss_and_grads(
        combination_image, base_image, style_reference_image
    )
    # 更新图像
    optimizer.apply_gradients([(grads, combination_image)])
    if i % 100 == 0:
        print(f"Iteration {i}: loss={loss:.2f}")
        img = deprocess_image(combination_image.numpy())
        fname = f"combination_image_at_iteration_{i}.png"
        # 保存图像
        keras.utils.save_img(fname, img)

Iteration 100: loss=8139.52
Iteration 200: loss=6653.05
Iteration 300: loss=6063.56
Iteration 400: loss=5742.18
Iteration 500: loss=5535.40
Iteration 600: loss=5389.46
Iteration 700: loss=5280.20
Iteration 800: loss=5194.65
Iteration 900: loss=5125.38
Iteration 1000: loss=5068.19
Iteration 1100: loss=5020.17
Iteration 1200: loss=4979.18
Iteration 1300: loss=4943.79
Iteration 1400: loss=4912.86
Iteration 1500: loss=4885.56
Iteration 1600: loss=4861.37
Iteration 1700: loss=4839.84
Iteration 1800: loss=4820.53
Iteration 1900: loss=4803.11
Iteration 2000: loss=4787.30
Iteration 2100: loss=4772.88
Iteration 2200: loss=4759.66
Iteration 2300: loss=4747.57
Iteration 2400: loss=4736.46
Iteration 2500: loss=4726.20
Iteration 2600: loss=4716.70
Iteration 2700: loss=4707.92
Iteration 2800: loss=4699.74
Iteration 2900: loss=4692.12
Iteration 3000: loss=4685.05
Iteration 3100: loss=4678.45
Iteration 3200: loss=4672.27
Iteration 3300: loss=4666.47
Iteration 3400: loss=4661.03
Iteration 3500: loss=46

### Wrapping up